In [21]:
import urllib.request
import re
import html
import os
from IPython.display import clear_output

# url начальной страницы газеты
newspaper_url = 'http://polkrug.ru/'

In [22]:
# функция для получения ответа с сервера по запросу url_request
def get_url_responce(url_request):
    
    # формируем запрос
    req = urllib.request.Request(url_request)
    
    try:
        with urllib.request.urlopen(req) as response:
            # получаем ответ
            html = response.read().decode('utf-8')
            return html
    
    # если что-то пошло не так, возвращаем сообщение
    except:
        return 'URL RESPONSE ERROR!'

In [23]:
# функция для получения информации об авторе статьи
def get_article_author(article_html):
    
    # регулярное выражение для поиска автора статьи
    author_re = re.compile('</span><span class="author_fio">(.*?)</span>')
    # результат поиска
    author_search = author_re.search(article_html)
    
    # если автор нашелся
    if author_search:
        # вернем его
        return author_search.group(1)
    
    # иначе вернем 'no_author'
    return 'no_author'

In [24]:
# функция для получения названия статьи
def get_article_title(article_html):
    
    # регулярное выражение для поиска названия статьи
    title_re = re.compile('<h1>(.+)</h1>')
    # результат поиска
    title_search = title_re.search(article_html)
    
    # если заголовок нашелся
    if title_search:
        # вернем его
        return title_search.group(1)
    
    # иначе вернем 'no_title'
    return 'no_title'

In [25]:
# функция для получения даты статьи
def get_article_date(article_html):
    
    # регулярное выражение для поиска даты статьи
    date_re = re.compile('<div class="news_date">(.+)</div>')
    # результат поиска
    date_search = date_re.search(article_html)
    
    # если дата не нашлась
    if not date_search:
        # вернем все это:
        return 'no_day', 'no_month', 'no_year'
    
    # если дата указана, осталось перевести ее в нужный формат
    date_split = date_search.group(1).split()
    dd_str_raw, mm_str_raw, yy_str = date_split[0], date_split[1], date_split[2]
    
    # переводи дни месяца в нужный формат (1 -> 01)
    if len(dd_str_raw) == 1:
        dd_str = '0' + dd_str_raw
    else:
        dd_str = dd_str_raw
    
    # числовые значения месяцев года
    month_dict = {'января': '01',
                  'февраля': '02',
                  'марта': '03',
                  'апреля': '04',
                  'мая': '05',
                  'июня': '06',
                  'июля': '07',
                  'августа': '08',
                  'сентября': '09',
                  'октября': '10',
                  'ноября': '11',
                  'декабря': '12'}
    
    # переводим названия месяцев в числовые обозначения
    mm_str = month_dict[mm_str_raw]
    
    return dd_str, mm_str, yy_str

In [26]:
# функция для получения категории и подкатегории (если есть) статьи
def get_article_topic(article_html):
    
    # регулярные выражения для поиска категории и подкатегории
    topic_re = re.compile('<div class="rubric_wrapper">\n\s+<a class="rubric" href="[^>]+">([^<]+)</a>')
    sub_topic_re = re.compile('<a class="podrubric" href=".+">(.+)</a>')
    
    # результат поиска категории
    topic_search = topic_re.search(article_html)
    
    # если категория указана, ОК
    if topic_search:
        topic = topic_search.group(1)
    else:
        # если нет, вернем вместо нее 'no topic'
        topic = 'no_topic'
    
    # результат поиска подкатегории
    sub_topic_search = sub_topic_re.search(article_html)
    
    # если нашли подкатегорию, добавим ее к категории через запятую и вернем это все
    if sub_topic_search:
        return topic + ', ' + sub_topic_search.group(1)
    
    # иначе вернем только категорию
    return topic

In [27]:
# функция для получения текста статьи
def get_article_text(article_html):
    
    # регулярное выражение для поиска текста статьи
    text_re = re.compile('<p>.+?</p>|<h3>.+?</h3>', flags= re.DOTALL)
    
    # результат поиска
    raw_text = text_re.findall(article_html)
    
    # список, в котором будут храниться абзацы текста
    final_text = []
    
    # регулярное выражение для оставшихся тегов и проблельных символов
    clean_re = re.compile('<.*?>|\n|\r|\t')
    
    for paragraph in raw_text:
        
        # очищаем абзац от оставшихся тегов, проблеьных символов и переводим все специальные символы в обычные
        clean_paragraph = html.unescape(clean_re.sub('', paragraph))
    
        # убираем строчку "нет комментариев", если она есть
        if clean_paragraph == 'Нет комментариев':
            continue
        
        # добавляем "очищенный" абзац к тексту
        final_text.append(clean_paragraph)
    
    return final_text

In [28]:
# функция для получения ссылки на главное изображение статьи
def get_main_image(article_html):
    
    # регулярное выражение для поиска главного изображения статьи
    main_image_re = re.compile('class="main_img" src="([^>]+)"')
    
    # результат поиска
    main_image_search = main_image_re.search(article_html)
    
    # если главное изображение нашлось, вернем его
    if main_image_search:
        return newspaper_url + main_image_search.group(1)
    
    # иначе вернем none
    return None

In [29]:
# функция для получения ссылок на изображения к статье
def get_article_images(article_html):
    
    # регулярное выражение для поиска изображений к статье
    image_re = re.compile('<a class="article_img" target="_blank" rel="gallery" href="([^>]+)">')
    
    # результат поиска
    image_urls = image_re.findall(article_html)
    
    # список url изображений
    image_urls_final = []
    
    # добавим url каждого изображения в список
    for url in image_urls:
        image_urls_final.append(newspaper_url + url)
    
    return image_urls_final

In [30]:
# функция для получения информации о статье и ее текста
def get_article(article_url):
    
    # получаем html-файл со статьей
    article_html = get_url_responce(newspaper_url + article_url)
    
    # заполняем словарь с информацией о статье
    article_info = {'au': get_article_author(article_html),
                    'ti': get_article_title(article_html),
                    'da': get_article_date(article_html),
                    'topic': get_article_topic(article_html),
                    'url': newspaper_url + article_url,
                    'main_img': get_main_image(article_html),
                    'img': get_article_images(article_html)}
    
    # получаем текст статьи
    article_text = get_article_text(article_html)
    
    return article_info, article_text

In [31]:
# функция для записи чего-то, связанного со статьей, в файл
def write_content(content, article_info, file_type):
    
    # словарь с расширениями
    ext_dict = {'html': '.html',
                'mystem-plain': '.txt'}
    
    # директория, в которую будет сохраняться файл
    dirname = os.path.join('.', 'Газета', file_type, article_info['da'][2], article_info['da'][1])
    
    # если она не существует, создадим
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    
    # имя файла = название статьи + нужное расширение
    # полное имя файла для статьи = директория + имя файла
    file_name = os.path.join(dirname, article_info['ti'] + ext_dict[file_type])
    
    # записываем в файл статью в режиме для чтения
    with open(file_name, 'w', encoding='utf-8') as my_file:
        my_file.write(content)
    
    # возвращаем полное имя файла - оно пригодится нам для mystem
    return file_name

In [32]:
def make_mystem_analysis(article_info, article_text):
    
    # имя директории для файла с резульатом mystem-palin
    dirname_plain = os.path.join('.', 'Газета', 'mystem-plain', article_info['da'][2], article_info['da'][1])
    
    # если нужно, создаем ее
    if not os.path.exists(dirname_plain):
        os.makedirs(dirname_plain)
    
    # имя директории для файла с результатом mystem-xml
    dirname_xml = os.path.join('.', 'Газета', 'mystem-xml', article_info['da'][2], article_info['da'][1])
    
    # если нужно, создаем ее
    if not os.path.exists(dirname_xml):
        os.makedirs(dirname_xml)
    
    # имя файла с результатом mystem-plain
    filename_plain = os.path.join(dirname_plain, article_info['ti'].replace(' ', '_') + '.txt')
    
    # имя файла с результатом mystem-xml
    filename_xml = os.path.join(dirname_xml, article_info['ti'].replace(' ', '_') + '.xml')
    
    # записываем текст статьи во временный файл
    with open('buffer.txt', 'w', encoding='utf-8') as my_file:
        my_file.write(' '.join(article_text))
    
    # вызываем mystem для plain
    os.system('./mystem -cdinl --eng-gr buffer.txt ' + filename_plain)
    
    # вызываем mystem для xml
    os.system('./mystem -cdinl --eng-gr --format xml buffer.txt ' + filename_xml)

In [33]:
def make_distilled_html(article_info, article_text):
    
    # открываем html заготовку, из которой будем делать статью в режиме для чтения
    with open('template.html', 'r', encoding='utf-8') as source_file:
        html_template = source_file.read()
    
    # заменяем ее части на то, что нужно
    html_template = html_template.replace('article_title', article_info['ti'])
    html_template = html_template.replace('author_name', article_info['au'])
    html_template = html_template.replace('article_date', '.'.join(article_info['da']))
    html_template = html_template.replace('article_url', article_info['url'])
    html_template = html_template.replace('article_topic', article_info['topic'])
    html_template = html_template.replace('article_url', article_info['url'])
    
    # будущий текст html-файла для чтения
    text_html = ''
    
    # вставляем главное изображение, если есть
    if article_info['main_img']:
        text_html += '<center><img src="' + article_info['main_img'] +  '"></center>'
    
    # собираем из параграфов текст и вставляем его
    for paragraph in article_text:
        text_html += '<p>' + paragraph + '</p>'
    
    html_template = html_template.replace('article_text', text_html)
    
    # часть html-файла, в которой будут картинки
    images_html = ''
    
    # дописываем "Фото к статье:", если есть картинки
    if len(article_info['img']) > 0:
        images_html += '<br/>Фото к статье:<br/><br/>'
    
    # вставляем картинки к статье, если есть
    for image in article_info['img']:
        images_html += '<center><img src="' + image + '"></center><br/><br/>'
    html_template = html_template.replace('image_block', images_html)
    
    # записываем получившийся html-файл куда нужно 
    write_content(html_template, article_info, 'html')

In [34]:
def get_metadata_line(article_info):
    
    # строчка для мета-таблицы
    row = '%s\t%s\t\t\t%s\t%s\tпублицистика\t\t\t%s\t\tнейтральный\tн-возраст\tн-уровень\t' +\
        'городская\t%s\t"Полярный круг"\t\t%s\tгазета\tРоссия\tЯНАО\tru'
    
    # заполним ее!
    metadata_line = row % (article_info['url'], article_info['au'], article_info['ti'], '.'.join(article_info['da']),
                           article_info['topic'], article_info['url'], article_info['da'][2])
    
    # мы считаем, что "path -- это путь к чистому неразмеченному файлу со статьёй" - это url страницы
    # иначе нужно было бы создавать отдельную папку с неразмеченным текстом на комрьютере,
    # а этого нет в структуре каталогов в описании проекта
    
    # и вернем
    return metadata_line

In [35]:
# функция для подсчета количества слов в тексте
def get_text_word_count(article_text):
    
    # счетчик слов
    word_cnt = 0
    
    # для каждого абзаца в тексте
    for paragraph in article_text:
        # добавляем его количество слов к счетчику слов
        word_cnt += len(paragraph.split())
    
    return word_cnt

In [36]:
# функция для сохранения страницы с режимом для чтения, вызова mystem и сохранения размеченного текста в формате XML
def handle_article_page(article_info, article_text):
    
    # посчитаем, сколько слов содержит статья
    word_cnt = get_text_word_count(article_text)
    
    # создадим html-файл со статьей в режиме для чтения
    make_distilled_html(article_info, article_text)
    
    # произведем анализ mystem'ом
    make_mystem_analysis(article_info, article_text)
    
    # получим строчку мета-таблицы
    metadata_line = get_metadata_line(article_info)
    
    return metadata_line, word_cnt

In [37]:
# функция для получения url следующей статьи газеты
def get_next_page_url(now_page_url):
    
    # получим html-файл текущей страницы
    now_html = get_url_responce(newspaper_url + now_page_url)
    
    # регулярное выражение для поиска ссылки на следующую страницу
    next_re = re.compile('<li class="next"><a href="([^"]+)">')
    
    # результат поиска
    next_page_url_search = next_re.search(now_html)
    
    # если следующая страница нашлась, вернем ее url
    if next_page_url_search:
        return next_page_url_search.group(1)
    
    # иначе вернем None
    return None

In [38]:
# функция для получения url статей на странице
def get_page_articles(page_url):
    
    # получим html-файл текущей страницы
    page_html = get_url_responce(newspaper_url + page_url)
    
    # регулярное выражение для поиска ссылок на статьи
    article_re = re.compile('<h3>\s*<a href="([^"]+)">')
    
    # результат поиска
    articles = article_re.findall(page_html)
    
    return articles

In [39]:
# главная функция - она начинает со страницы start_url, ищет на ней статьи, обрабатывает их,
# а потом делает то же самое для остальных страниц
def load_newspaper_articles(start_url):
    
    # начинаем с первой страницы
    now_page_url = start_url
    
    # счетчик слов обработанных статей
    total_word_cnt = 0
    
    # список строчек мета-таблицы
    metadata_list = []
    
    # пока страницы газеты в нужной рубрике не закончились, выкачиваем с них статьи и обрабатываем их
    while now_page_url:
        
        # сотрем то, что было до этого в выводе
        clear_output()
        
        # печатаем url текущей страницы газеты, чтобы видеть, что программа жива
        print(now_page_url)
        
        # получаем url статей текущей страницы
        now_page_article_urls = get_page_articles(now_page_url)  
        
        # для каждой статьи из списка
        for article_url in now_page_article_urls:
            
            # обрабатываем ee и получаем строчки мета-таблицы
            metadata_line, word_cnt = handle_article_page(*get_article(article_url))
            
            # записываем строку в мета-таблицу
            metadata_list.append(metadata_line)
            
            # добавляем количество слов статьи к полному количеству слов
            total_word_cnt += word_cnt
        
        # получаем url следующей страницы газеты
        now_page_url = get_next_page_url(now_page_url)
        
    # записываем мета-таблицу в файл
    with open(os.path.join('.', 'Газета', 'metadata.csv'), 'w', encoding='utf-8') as metadata_file:
        metadata_file.write('\n'.join(metadata_list))
    
    # печатаем, сколько слов обработали
    print('Суммарный объем статей: ' + str(total_word_cnt) + ' слов')

In [40]:
# вызываем главную функцию от раздела "Культура" газеты "Полярный круг"
load_newspaper_articles('news/kultura')

/news/kultura?p=72
Суммарный объем статей: 325655 слов
